In [56]:
"""Import the required modules"""
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

Analyzing welding data by welding types

In [57]:
df2 = pd.read_csv('welding_data_by_welding_types.csv')
df2 = pd.get_dummies(df2).drop('welding_type_GMAW', axis=1)
df2 = df2[['current', 'voltage', 'welding_speed', 'plate_thickness',	'welding_type_SAW',	'welding_type_SMAW',	              'welding_type_GTAW', 'flexural_strength', 'hardness_value', 'residual_stress',
           'tensile_strength']]
df2.head()

,current,voltage,welding_speed,plate_thickness,welding_type_SAW,welding_type_SMAW,welding_type_GTAW,flexural_strength,hardness_value,residual_stress,tensile_strength
0,160,32,120,6,0,0,1,236.2,102.5,152.0,432.6
1,130,32,90,8,0,1,0,270.2,96.4,270.2,332.8
2,100,28,120,8,0,0,1,360.8,86.4,90.5,382.4
3,160,28,90,10,0,0,1,230.0,105.0,165.4,422.8
4,100,28,120,8,1,0,0,276.5,128.4,186.6,385.5


Exploratory data analysis

In [58]:
# pos = 1
# for i in range(4):
#   print(f'\t{df2.columns[i].capitalize()} vs Response values')
#   plt.figure()
#   for j in range(8, 11):
#     plt.subplot(1, 3, pos)
#     plt.scatter(df2.iloc[:, i], df2.iloc[:, j])
#     plt.ylabel(df2.columns[j])
#     plt.xlabel(df2.columns[i])
#     pos += 1
#   plt.tight_layout()
#   plt.show()
#   print('\n')
#   pos = 1

Generate the necessary variables for modeling

In [59]:
X1 = df2.iloc[:, :7]
X = X1.to_numpy()
y1 = df2.iloc[:, -4:]
y = y1.to_numpy()

In [60]:
def relu(x):
   return np.maximum(x, 0, x)

In [61]:
def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = relu(G)
    return H

In [62]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

In [94]:
def MAD(y, y_pred):
    '''Calculates the Mean Absolute Deviation of two arrays of numbers
    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    return np.average(np.abs(y - y_pred))

In [98]:
def MAPE(y, y_pred):
    '''
    Calculates the relationship between the average prediction error
    and the average expected target error for several consecutive periods.

    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    # Calculate prediction error
    error = y - y_pred

    return np.average((np.abs(error)/y) * 100)

In [64]:
# Split data in readiness for training.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

# Convert test data to dataframe and print it.
y_test_df = pd.DataFrame(y_test, columns=['flexural_strength',	'hardness_value',	'residual_stress',	'tensile_strength'])

In [65]:
Predicting individual response separately to optimize prediction

For 5 nodes
______________________


IndexError: tuple index out of range

# print('Expected Data')
# print(y_test_df)
# print('\n')

# Run ELM for 5, 10, 15, 20, 25, 30 nodes.
metrics_table = []
for hidden_size in range(5, 35, 5):
  print(f'For {hidden_size} nodes')
  print('______________________')
  for i in range(y.shape[1]):
    cols = y[:, i]
    X_train, X_test, y_train, y_test = train_test_split(X, cols,random_state=1, test_size=0.2)
    # ELM Calculations.
    e_tic = time.perf_counter() #Start timer
    input_size = X_train.shape[1] #Set input size to the number of columns in the X train data
    input_weights = np.random.normal(size=[input_size,hidden_size]) #Create input weight matrix
    biases = np.random.normal(size=[hidden_size]) #Set up biases the size of hidden layer
    output_weights = np.dot(pinv2(hidden_nodes(X_train)), y_train)
    prediction = predict(X_test)  #Predict
    # Convert prediction array to dataframe
    # prediction_df = pd.DataFrame(prediction, columns=['flexural_strength',	'hardness_value',	'residual_stress',	'tensile_strength'])
    e_toc = time.perf_counter() #Stop timer

    # for i in range(y_test.shape[1]):
    #   print(f'Prediction')
    metrics_row = [hidden_size]
    for i in range(y_test.shape[1]):
      metrics_row.append(r2_score(prediction[:, i], y_test[:, i]))
      # print(f'R^2 for {prediction_df.columns[i]}: {(r2_score(prediction[:, i], y_test[:, i]))}')
    
    metrics_table.append(metrics_row)
    #   # print(metrics_row)

    metrics_table_df = pd.DataFrame(metrics_table, columns=[['Nodes'] + list(prediction_df.columns)])

print(metrics_table_df)
  # display(prediction_df)

  

    # # ANN calculations
    # a_tic = time.perf_counter()
    # sc_X = StandardScaler()
    # X_trainscaled=sc_X.fit_transform(X_train)
    # X_testscaled=sc_X.transform(X_test)
    # reg = MLPRegressor(hidden_layer_sizes=(h,h,h),activation="relu" ,random_state=1, max_iter=1000000).fit(X_trainscaled, y_train)
    # y_pred=reg.predict(X_testscaled)
    # a_tic = time.perf_counter()

    # print(f'Metrics for Response variable, {y1.columns[i].capitalize()}:')

    # print(f'E_Exp\t\tE_Pre\t\tA_Pre')
    # for j in range(len(y_test)):
    #   print(f'{y_test[j]}\t{prediction[j]}\t{y_pred}')

    # print(f'R^2: {(r2_score(prediction, y_test))}')
    # print(f'rmse: {np.sqrt(mean_squared_error(y_test, prediction))}')
    # print(f'mae: {(mean_absolute_error(y_test, prediction))}\n')
    # print(f'Time: {')

In [100]:
metrics
for hidden_size in range(5, 6, 5):
    metric = [hidden_size]
    print(f'\nELM Analysis for {hidden_size} Node(s)')
    for i in range(y.shape[1]):
        cols = y[:, i]
        X_train, X_test, y_train, y_test = train_test_split(X, cols,random_state=1, test_size=0.2)

        input_size = X_train.shape[1]
        # hidden_size = 5000
        input_weights = np.random.normal(size=[input_size,hidden_size])
        biases = np.random.normal(size=[hidden_size])
        output_weights = np.dot(pinv2(hidden_nodes(X_train)), y_train)
        prediction = predict(X_test)

        # Calculate the necessary metrics
        r2 = r2_score(prediction, y_test)
        mse = mean_squared_error(y_test, prediction)
        rmse = np.sqrt(mse)
        mad = MAD(y_test, prediction)
        mape = MAPE(y_test, prediction)


        print(f'{y1.columns[i].capitalize()}:')
        print(f'\tR^2: {(r2_score(prediction, y_test))}')
        print(f'\trmse: {np.sqrt(mean_squared_error(y_test, prediction))}')


ELM Analysis for 5 Node(s)
Flexural_strength:
	R^2: -5.505226305243115
	rmse: 74.75868761609773
Hardness_value:
	R^2: -5.595702374554594
	rmse: 29.34272984300399
Residual_stress:
	R^2: -0.2459055354589379
	rmse: 72.3554404573569
Tensile_strength:
	R^2: -3.195076028605933
	rmse: 88.91082516518166


In [ ]:
def TS()